In [1]:
import numpy as np
import matplotlib.pyplot as plt

from preprocessing.preprocessor import Preprocess     #  From usatellite
from models.unet2d.unet2d_model import Unet2d         #  From usatellite
from plot_utils.plotting_utils import SatPlotter      #  From usatellite
from loss.loss_metrics import f1
from loss.loss_metrics import categorical_focal_loss
from loss.loss_metrics import weighted_categorical_crossentropy

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from skimage.transform import resize

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

#TEMP
import tensorflow as tf

In [2]:
aug_imgs = np.load(r'C:\\Users\\Cody\\Documents\\usatellite_data\\aug_imgs.npy')
aug_labels = np.load(r'C:\\Users\\Cody\\Documents\\usatellite_data\\aug_labels.npy')

In [3]:
preproc = Preprocess()

In [6]:
imgs_array_scaled = preproc.unit_normalize_dims(aug_imgs)

In [7]:
x_train, x_val, y_train, y_val = train_test_split(imgs_array_scaled, aug_labels, test_size=0.30, shuffle=True, random_state=42)

In [8]:
unet2dmodel = Unet2d(input_img=imgs_array_scaled)

In [9]:
unet = unet2dmodel.get_unet()

In [10]:
unet.compile(optimizer=Adam(), loss=weighted_categorical_crossentropy, metrics=["categorical_accuracy", f1])

In [ ]:
results = unet.fit(x_train, y_train, batch_size=10, epochs=200, validation_data=(x_val, y_val))

Epoch 1/200
9/9 [==============================] - 270s 30s/step - loss: 0.0419 - categorical_accuracy: 0.2557 - f1: 0.2452 - val_loss: 0.1256 - val_categorical_accuracy: 0.2324 - val_f1: 0.1269
Epoch 2/200
9/9 [==============================] - 271s 30s/step - loss: 0.0379 - categorical_accuracy: 0.3570 - f1: 0.3261 - val_loss: 0.2037 - val_categorical_accuracy: 0.2386 - val_f1: 0.1331
Epoch 3/200
4/9 [============>.................] - ETA: 1:57 - loss: 0.0407 - categorical_accuracy: 0.3871 - f1: 0.3702

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["f1"], label="f1")
plt.plot(results.history["val_f1"], label="val_f1")
#plt.plot( np.argmin(results.history["val_f1"]), np.min(results.history["val_f1"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

In [ ]:
test_imgs = unet.predict(x_val)

In [ ]:
test_classes = np.argmax(test_imgs, -1)

In [ ]:
satplot = SatPlotter()

In [ ]:
satplot.plot_image_and_result(x_val, y_val, test_classes)